# Data Cleaned

In [30]:
import numpy as np
import pandas as pd

In [31]:
df_vacc = pd.read_csv('child_vaccination_data.csv')
num_rows_original = df_vacc.shape[0]
df_vacc.shape

(118423, 10)

In [32]:
df_vacc.head()

,Vaccine,Dose,Geography Type,Geography,Birth Year/Birth Cohort,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
0,Influenza,NaN,States/Local Areas,Tennessee,2014-2017,Poverty,>400% FPL,64.2,55.3 to 73.1,189.0
1,Hep A,≥1 Dose,States/Local Areas,Tennessee,2016-2019,Race and Ethnicity,Hispanic,86.4,76.8 to 95.9,187.0
2,PCV,≥4 Doses,States/Local Areas,Tennessee,2016-2019,Insurance Coverage,Private Insurance Only,89.3,85.4 to 93.2,612.0
3,Influenza,NaN,States/Local Areas,Tennessee,2014-2017,Poverty,133% to <400% FPL,51.4,44.0 to 58.8,342.0
4,Influenza,NaN,States/Local Areas,Tennessee,2014-2017,Poverty,<133% FPL,39.0,31.9 to 46.1,298.0


In [33]:
# pivot 'Dimension Type' into new columns (with 'Dimension' containing values for new columns)
df_vacc = pd.pivot(df_vacc, 
                   index=['Vaccine', 'Dose', 'Geography Type', 'Geography', 'Birth Year/Birth Cohort', \
                          'Dimension', 'Estimate (%)', '95% CI (%)', 'Sample Size'], # identifier for each row
                   columns=['Dimension Type'], # column containing new column names
                   values='Dimension')         # values for new columns

df_vacc.reset_index(inplace=True) 
df_vacc.rename_axis(None, axis=1, inplace=True) # remove the named index

In [34]:
# drop 'Dimension' column as it is no longer needed after pivot
df_vacc.drop(columns='Dimension', inplace = True)

In [35]:
# split 'Birth Year/Birth Cohort' column into two columns by looking for hyphen in values
df_vacc['Birth Year'] = np.where(df_vacc['Birth Year/Birth Cohort']\
                                 .str.contains('-'), np.nan, df_vacc['Birth Year/Birth Cohort'])
df_vacc['Birth Cohort'] = np.where(df_vacc['Birth Year/Birth Cohort']\
                                   .str.contains('-'), df_vacc['Birth Year/Birth Cohort'], np.nan)

In [36]:
# drop 'Birth Year/Birth Cohort' column as it is no longer needed after split
df_vacc.drop(columns='Birth Year/Birth Cohort', inplace = True)

In [37]:
# reorder columns
df_vacc = df_vacc[['Vaccine', 'Dose', 'Geography Type', 'Geography', 'Birth Year', 'Birth Cohort', \
                   'Estimate (%)', '95% CI (%)', 'Overall', 'Age', 'Race and Ethnicity', 'Poverty', \
                    'Insurance Coverage', 'Urbanicity', 'Sample Size']]

In [38]:
# check for NA in Estimate (%) which will be column needed in all data visualizations in app
df_vacc['Estimate (%)'].isna().sum()

58

In [39]:
# drop rows with NA in Estimate (%) which will be column needed in all data visualizations in app
df_vacc.dropna(subset=['Estimate (%)'], inplace=True)

num_rows_current = df_vacc.shape[0]

print(num_rows_original - num_rows_current, "rows dropped")

58 rows dropped


In [40]:
# fill NA values for Dose for certain vaccines (based on CDC documentation of what these represent)
df_vacc['Dose'] = np.where(df_vacc['Vaccine'] == 'Combined 7 Series', 'Full Series', df_vacc['Dose'])
df_vacc['Dose'] = np.where(df_vacc['Vaccine'] == 'Influenza', '≥2 Doses at least 24 days apart', df_vacc['Dose'])
df_vacc['Dose'] = np.where(df_vacc['Vaccine'] == 'Rotavirus', 'Full Series', df_vacc['Dose'])

In [41]:
# update values for HHS Regions to actually include 'HHS' in value to be more clear what Region means
df_vacc['Geography'] = np.where(df_vacc['Geography'].str.contains('Region'), \
                                df_vacc['Geography'].str.replace('Region', 'HHS Region'), df_vacc['Geography'])

In [42]:
# rename Geography column to be more clear for use in app
df_vacc.rename(columns={'Geography': 'Geographic Area'}, inplace=True)

In [43]:
df_vacc.sample(10)

,Vaccine,Dose,Geography Type,Geographic Area,Birth Year,Birth Cohort,Estimate (%),95% CI (%),Overall,Age,Race and Ethnicity,Poverty,Insurance Coverage,Urbanicity,Sample Size
96996,PCV,≥4 Doses,States/Local Areas,PA-Philadelphia,2018,NaN,87.8,80.7 to 93.2,NaN,35 Months,NaN,NaN,NaN,NaN,224.0
102161,Polio,≥2 Doses,States/Local Areas,New York,NaN,2014-2015,90.3,87.9 to 92.3,NaN,7 Months,NaN,NaN,NaN,NaN,1095.0
43298,Hep B,≥3 Doses,States/Local Areas,Georgia,NaN,2016-2019,83.4,68.9 to 97.9,NaN,NaN,NaN,NaN,Other,NaN,57.0
69126,Hib,≥3 Doses,States/Local Areas,NY-City of New York,2013,NaN,88.4,86.1 to 90.4,NaN,19 Months,NaN,NaN,NaN,NaN,279.0
101502,Polio,≥2 Doses,States/Local Areas,Massachusetts,NaN,2015-2016,96.5,93.6 to 98.1,NaN,13 Months,NaN,NaN,NaN,NaN,559.0
36932,Hep B,"≥1 Dose, 3 Day (Birth Dose)",States/Local Areas,TX-El Paso County,NaN,2016-2019,78.9,65.5 to 92.2,NaN,NaN,NaN,NaN,Private Insurance Only,NaN,60.0
791,Combined 7 Series,Full Series,States/Local Areas,Connecticut,NaN,2014-2017,77.6,72.7 to 82.5,NaN,NaN,NaN,NaN,NaN,Living In a MSA Non-Principal City,478.0
90205,PCV,≥3 Doses,States/Local Areas,New York,2011,NaN,90.5,86.3 to 93.5,NaN,19 Months,NaN,NaN,NaN,NaN,466.0
66057,Hib,≥3 Doses,HHS Regions/National,HHS Region 4,2018,NaN,92.8,90.0 to 95.0,NaN,35 Months,NaN,NaN,NaN,NaN,2317.0
36421,Hep B,"≥1 Dose, 3 Day (Birth Dose)",States/Local Areas,Ohio,NaN,2016-2019,70.9,65.7 to 76.1,NaN,NaN,NaN,NaN,NaN,Living In a MSA Principal City,455.0


In [44]:
# saved cleaned data to csv
df_vacc.to_csv('child_vaccination_data_cleaned.csv', index=False)